<a href="https://colab.research.google.com/github/ank090/Predictive-model-Car-Evaluation/blob/main/Car_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install category_encoders

In [17]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC  

In [18]:
dataset=pd.read_csv('cars.csv')
x=dataset.iloc[:,1:-1]
y=dataset.iloc[:,-1]

In [19]:
x.iloc[:,0:2].replace({'vhigh':3,'high':2,'med':1,'low':0},inplace=True) 
x.iloc[:,-2].replace({'small':0,'med':1,'big':2},inplace=True)
x.iloc[:,-1].replace({'low':0,'med':1,'high':2},inplace=True)
#ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[2])],remainder='passthrough')
oe=OneHotEncoder(cols=['persons','doors'])
x=oe.fit_transform(x)
x=x.drop(['persons_1','doors_1'],axis='columns')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


Splitting data into training and test dataset

In [20]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain

,buying,maint,doors_2,doors_3,doors_4,persons_2,persons_3,lug_boot,safety
107,3,3,0,0,1,0,1,2,2
901,1,3,1,0,0,1,0,0,1
1709,0,0,0,0,1,0,0,2,2
706,2,1,0,1,0,0,0,1,1
678,2,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
1130,1,1,1,0,0,0,1,1,2
1294,1,0,0,0,1,0,1,2,1
860,2,0,0,0,1,0,1,1,2
1459,0,2,0,1,0,0,0,0,1


Feature scaling

In [21]:
sc=StandardScaler()
xtrain.iloc[:,:2]=sc.fit_transform(xtrain.iloc[:,:2])
xtest.iloc[:,:2]=sc.fit_transform(xtest.iloc[:,:2])
xtrain.iloc[:,-2:]=sc.fit_transform(xtrain.iloc[:,-2:])
xtest.iloc[:,-2:]=sc.fit_transform(xtest.iloc[:,-2:])
xtrain

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,buying,maint,doors_2,doors_3,doors_4,persons_2,persons_3,lug_boot,safety
107,1.360492,1.349331,0,0,1,0,1,1.224748,1.23546
901,-0.432736,1.349331,1,0,0,1,0,-1.221208,0.01239
1709,-1.329350,-1.362412,0,0,1,0,0,1.224748,1.23546
706,0.463878,-0.458498,0,1,0,0,0,0.001770,0.01239
678,0.463878,-0.458498,1,0,0,0,0,0.001770,-1.21068
...,...,...,...,...,...,...,...,...,...
1130,-0.432736,-0.458498,1,0,0,0,1,0.001770,1.23546
1294,-0.432736,-1.362412,0,0,1,0,1,1.224748,0.01239
860,0.463878,-1.362412,0,0,1,0,1,0.001770,1.23546
1459,-1.329350,0.445417,0,1,0,0,0,-1.221208,0.01239


Training the calssification models on training set



1) Knn

In [22]:
#print("select the model to use or check the scores of each \n 1) knn\n 2) Random Forest\n 3) Naive bayes\n 4) Decision tree\n 5) Logistic regression/n 6) SVC(Support vector classificaion)")
knn=KNeighborsClassifier(n_neighbors=5,metric='minkowski')
knn.fit(xtrain,ytrain)
ypred_knn=knn.predict(xtest)
cs=classification_report(ytest,ypred_knn)
print(cs)

              precision    recall  f1-score   support

         acc       0.84      0.78      0.81        83
        good       0.38      0.45      0.42        11
       unacc       0.94      0.99      0.96       235
       vgood       1.00      0.41      0.58        17

    accuracy                           0.90       346
   macro avg       0.79      0.66      0.69       346
weighted avg       0.90      0.90      0.89       346



2) Random forest

In [23]:
rf=RandomForestClassifier(n_estimators=100,criterion='entropy')
rf.fit(xtrain,ytrain)
ypred_rf=rf.predict(xtest)
cs=classification_report(ytest,ypred_rf)
print(cs)

              precision    recall  f1-score   support

         acc       0.96      0.81      0.88        83
        good       0.45      0.91      0.61        11
       unacc       0.98      1.00      0.99       235
       vgood       0.86      0.71      0.77        17

    accuracy                           0.94       346
   macro avg       0.81      0.86      0.81       346
weighted avg       0.95      0.94      0.94       346



3) Naive Bayes

In [24]:
nb=GaussianNB()
nb.fit(xtrain,ytrain)
ypred_nb=nb.predict(xtest)
cs=classification_report(ytest,ypred_nb)
print(cs)

              precision    recall  f1-score   support

         acc       0.57      0.43      0.49        83
        good       0.40      0.36      0.38        11
       unacc       0.82      0.96      0.89       235
       vgood       0.00      0.00      0.00        17

    accuracy                           0.77       346
   macro avg       0.45      0.44      0.44       346
weighted avg       0.71      0.77      0.73       346



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


4) Decision tree

In [25]:
dt=DecisionTreeClassifier(criterion='entropy')
dt.fit(xtrain,ytrain)
ypred_dt=dt.predict(xtest)
cs=classification_report(ytest,ypred_dt)
print(cs)

              precision    recall  f1-score   support

         acc       0.97      0.80      0.87        83
        good       0.48      0.91      0.62        11
       unacc       0.98      1.00      0.99       235
       vgood       0.83      0.88      0.86        17

    accuracy                           0.94       346
   macro avg       0.82      0.90      0.84       346
weighted avg       0.96      0.94      0.95       346



5) Logistic regression

In [26]:
lr=LogisticRegression()
lr.fit(xtrain,ytrain)
ypred_lr=lr.predict(xtest)
cs=classification_report(ytest,ypred_lr)
print(cs)

              precision    recall  f1-score   support

         acc       0.73      0.71      0.72        83
        good       0.45      0.45      0.45        11
       unacc       0.91      0.94      0.93       235
       vgood       0.82      0.53      0.64        17

    accuracy                           0.85       346
   macro avg       0.73      0.66      0.69       346
weighted avg       0.85      0.85      0.85       346



6) Support vector classification

In [27]:
sv=SVC(C=1.0,kernel='rbf',degree=3)
sv.fit(xtrain,ytrain)
ypred_sv=sv.predict(xtest)
cs=classification_report(ytest,ypred_sv)
print(cs)

              precision    recall  f1-score   support

         acc       0.97      0.82      0.89        83
        good       0.56      0.82      0.67        11
       unacc       0.97      1.00      0.99       235
       vgood       0.83      0.88      0.86        17

    accuracy                           0.95       346
   macro avg       0.83      0.88      0.85       346
weighted avg       0.95      0.95      0.95       346

